In [ ]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import random
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

In [ ]:
np.set_printoptions(suppress=True) # e 표기법 중지
np.set_printoptions(precision=2) # 소수점 2자리까지 표시

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

## 확인하려는 config & checkpoint 경로 입력

In [ ]:
# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/mmdetection_trash/configs/trash/ineedyolo/default.py') # config파일 경로
# checkpoint path
cfg.work_dir = '/opt/ml/code/mmdetection_trash/work_dirs/step2' 
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_100.pth') # .pth파일 경로

In [ ]:
PREFIX = '/opt/ml/input/data/'

epoch = 12

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]

# cfg.model.roi_head.bbox_head.num_classes = 11 # 모델마다 사용여부가 다르니 주의 !!!

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [ ]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = classes
model = MMDataParallel(model.cuda(), device_ids=[0])

## Inference on validation data

In [ ]:
valid_dataset = build_dataset(cfg.data.val)
data_loader = build_dataloader(valid_dataset, samples_per_gpu=1, workers_per_gpu=cfg.data.workers_per_gpu, dist=False, shuffle=False)

valid_coco = COCO("/opt/ml/input/data/val.json")
# valid_coco = COCO("../../input/data/val.json")

valid_output = single_gpu_test(model, data_loader, show_score_thr=0.05)

## Define for helper function

In [ ]:
class_num = 11
class_name = classes = (
    "UNKNOWN", "General trash", "Paper", "Paper pack", 
    "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", 
    "Battery", "Clothing"
)
COLORS = [
    (204,24,30),
    (39,147,232),
    (85,153,0),
    (102,102,102),
    (27,133,184),
    (90,82,85),
    (85,158,131),
    (174,90,65),
    (195,203,113),
    (243,119,54),
    (184,167,234)
]

def bbox_on_image(image, bboxes):
    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image)
    
    for cls_idx in range(class_num):
        for cls_bbox in bboxes[cls_idx]:
            cls_name = class_name[cls_idx]
            cls_score = cls_bbox[-1]
            xmin, ymin, xmax, ymax = map(lambda x: int(x), cls_bbox[:4])
            if cls_score > 0.05:
                draw.rectangle(((xmin, ymin), (xmax, ymax)), outline=COLORS[cls_idx])
                draw.text((xmin, ymin), f"{cls_name}\n{cls_score:4f}", fill="#000000")
    
    return image

def get_gt_bboxes(anns):
    gt_bboxes = [[] for _ in range(class_num)]
    
    for ann in anns:
        cls_id = ann["category_id"]
        xmin, ymin, w, h = [int(v) for v in ann["bbox"]]
        xmax, ymax = xmin + w, ymin + h
        
        gt_bboxes[cls_id].append([xmin, ymin, xmax, ymax, 1.0])
        
    return gt_bboxes

## Compare predicted image with ground truth

In [ ]:
idx = random.randint(0, 655) # 확인하려는 이미지
# idx = 0

image = cv2.imread(os.path.join("/opt/ml/input/data", valid_coco.load_imgs(idx)[0]["file_name"]))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image.copy()

gt_bboxes = get_gt_bboxes(valid_coco.load_anns(valid_coco.get_ann_ids(idx)))
# print(gt_bboxes)
# print(type(gt_bboxes))
# print(valid_output[idx][1:])
# print(type(valid_output[idx])
pred_bboxes = valid_output[idx]

gt_image = bbox_on_image(image, gt_bboxes)
pred_image = bbox_on_image(image, pred_bboxes)

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.title(valid_coco.load_imgs(idx)[0]["file_name"] + " - GT")
plt.axis("off")
plt.imshow(gt_image)

plt.subplot(1, 2, 2)
plt.title(valid_coco.load_imgs(idx)[0]["file_name"] + " - Predicted")
plt.axis("off")
plt.imshow(pred_image)

plt.show()

## Inference on test data

In [ ]:
test_dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(test_dataset, samples_per_gpu=1, workers_per_gpu=cfg.data.workers_per_gpu, dist=False, shuffle=False)

test_coco = COCO("/opt/ml/input/data/test.json")
test_output = single_gpu_test(model, data_loader, show_score_thr=0.05)

## Draw predicted test image

In [ ]:
idx = random.randint(0, 655) # 확인하려는 이미지
# idx = 6

image = cv2.imread(os.path.join("/opt/ml/input/data", test_coco.load_imgs(idx)[0]["file_name"]))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image.copy()
bboxes = test_output[idx]

bboxed_image = bbox_on_image(image, bboxes)

plt.figure(figsize=(12, 12))
plt.axis("off")
plt.title(test_coco.load_imgs(idx)[0]["file_name"])
plt.imshow(bboxed_image)
plt.show()